In [1]:
from geoband.API import *
import folium
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

/opt/app-root/lib/python3.6/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
df_gid = gpd.read_file('32.수원시_표준격자(250M_x_250M).geojson')
df_gid

,gid,geometry
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ..."
1,다사49aa19ba,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ..."
2,다사49aa20aa,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ..."
3,다사49ab18ab,"MULTIPOLYGON (((126.92766 37.26177, 126.92764 ..."
4,다사49ab18ba,"MULTIPOLYGON (((126.92764 37.26402, 126.92762 ..."
...,...,...
2085,다사63ba21aa,"MULTIPOLYGON (((127.08822 37.28722, 127.08821 ..."
2086,다사63ba21ab,"MULTIPOLYGON (((127.08821 37.28947, 127.08820 ..."
2087,다사63ba21ba,"MULTIPOLYGON (((127.08820 37.29173, 127.08819 ..."
2088,다사63ba21bb,"MULTIPOLYGON (((127.08819 37.29398, 127.08818 ..."


In [3]:
df_cctv = pd.read_csv('24.4개시도_CCTV설치현황.csv')
df_cctv.head()

,address,purpose,cctv_cnt,set_yr,lat,lon
0,경상남도 진주시 집현면 지내리 125,어린이보호,1,`2013-12,35.241319,128.093399
1,경상남도 진주시 집현면 지내리 125,어린이보호,4,`2013-12,35.242023,128.091990
2,경상남도 진주시 집현면 지내리 125,어린이보호,4,`2013-12,35.241209,128.092923
3,경상남도 진주시 진성면 상촌리 541-2,어린이보호,1,`2013-12,35.169310,128.224780
4,경상남도 진주시 진성면 상촌리 541-2,어린이보호,4,`2013-12,35.169223,128.225446


In [4]:
df_cctv.dropna(subset=['address'], inplace=True)
df_cctv['address'].isnull().sum()

0

In [5]:
df_cctv[df_cctv['address'].str.startswith('팔달')]

,address,purpose,cctv_cnt,set_yr,lat,lon
36690,팔달구 남창동 117-2,쓰레기단속,1,`2020-05,37.278384,127.015372
36691,팔달구 중동 121-1,쓰레기단속,1,`2020-05,37.272549,127.016138


In [6]:
df_cctv_suwon = df_cctv[df_cctv['address'].str.startswith('경기') | df_cctv['address'].str.startswith('수원') 
                        | df_cctv['address'].str.startswith('팔달') | df_cctv['address'].str.startswith('권선')
                        | df_cctv['address'].str.startswith('장안') | df_cctv['address'].str.startswith('영통')]
df_cctv_suwon

,address,purpose,cctv_cnt,set_yr,lat,lon
34525,경기도 수원시 권선구 구운동 530,생활방범,3,`2020-04,37.275290,126.972731
34526,경기도 수원시 권선구 당수동 177-17,생활방범,5,`2020-04,37.293090,126.944104
34527,경기도 수원시 권선구 입북동 306,생활방범,3,`2020-04,37.297725,126.955908
34528,경기도 수원시 권선구 입북동 428,생활방범,4,`2020-04,37.293739,126.958844
34529,경기도 수원시 권선구 입북동 801-42,생활방범,4,`2020-04,37.295225,126.958319
...,...,...,...,...,...,...
38358,경기도 수원시 권선구 서둔동 17-225,생활방범,4,`2013-12,37.262912,126.993314
38359,경기도 수원시 권선구 서둔동 260-43,생활방범,4,`2013-12,37.278874,126.979325
38360,경기도 수원시 권선구 세류1동 223-2,생활방범,4,`2013-12,37.262486,127.008331
38361,경기도 수원시 권선구 입북동 236-3,생활방범,4,`2013-12,37.294098,126.960498


In [7]:
df_cctv.reindex(index=list(range(34525, 38363)))

,address,purpose,cctv_cnt,set_yr,lat,lon
34525,경기도 수원시 권선구 구운동 530,생활방범,3,`2020-04,37.275290,126.972731
34526,경기도 수원시 권선구 당수동 177-17,생활방범,5,`2020-04,37.293090,126.944104
34527,경기도 수원시 권선구 입북동 306,생활방범,3,`2020-04,37.297725,126.955908
34528,경기도 수원시 권선구 입북동 428,생활방범,4,`2020-04,37.293739,126.958844
34529,경기도 수원시 권선구 입북동 801-42,생활방범,4,`2020-04,37.295225,126.958319
...,...,...,...,...,...,...
38358,경기도 수원시 권선구 서둔동 17-225,생활방범,4,`2013-12,37.262912,126.993314
38359,경기도 수원시 권선구 서둔동 260-43,생활방범,4,`2013-12,37.278874,126.979325
38360,경기도 수원시 권선구 세류1동 223-2,생활방범,4,`2013-12,37.262486,127.008331
38361,경기도 수원시 권선구 입북동 236-3,생활방범,4,`2013-12,37.294098,126.960498


수원의 경우 address가 null인 것이 없다.

---

### 표준격자 별 CCTV 갯수 COUNT

In [8]:
df_cctv_suwon.reset_index(drop=True, inplace=True)
df_cctv_suwon

,address,purpose,cctv_cnt,set_yr,lat,lon
0,경기도 수원시 권선구 구운동 530,생활방범,3,`2020-04,37.275290,126.972731
1,경기도 수원시 권선구 당수동 177-17,생활방범,5,`2020-04,37.293090,126.944104
2,경기도 수원시 권선구 입북동 306,생활방범,3,`2020-04,37.297725,126.955908
3,경기도 수원시 권선구 입북동 428,생활방범,4,`2020-04,37.293739,126.958844
4,경기도 수원시 권선구 입북동 801-42,생활방범,4,`2020-04,37.295225,126.958319
...,...,...,...,...,...,...
3833,경기도 수원시 권선구 서둔동 17-225,생활방범,4,`2013-12,37.262912,126.993314
3834,경기도 수원시 권선구 서둔동 260-43,생활방범,4,`2013-12,37.278874,126.979325
3835,경기도 수원시 권선구 세류1동 223-2,생활방범,4,`2013-12,37.262486,127.008331
3836,경기도 수원시 권선구 입북동 236-3,생활방범,4,`2013-12,37.294098,126.960498


In [9]:
from shapely.geometry import Point, Polygon, LineString
points = []
for i in range(0, len(df_cctv_suwon)):
    point = Point(df_cctv_suwon['lon'][i], df_cctv_suwon['lat'][i])
    points.append(point)

points

 ...]

In [10]:
df_cctv_suwon['geometry'] = points
df_cctv_suwon

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,address,purpose,cctv_cnt,set_yr,lat,lon,geometry
0,경기도 수원시 권선구 구운동 530,생활방범,3,`2020-04,37.275290,126.972731,POINT (126.972731 37.27529000000001)
1,경기도 수원시 권선구 당수동 177-17,생활방범,5,`2020-04,37.293090,126.944104,POINT (126.944104 37.29309)
2,경기도 수원시 권선구 입북동 306,생활방범,3,`2020-04,37.297725,126.955908,POINT (126.955908 37.297725)
3,경기도 수원시 권선구 입북동 428,생활방범,4,`2020-04,37.293739,126.958844,POINT (126.958844 37.293739)
4,경기도 수원시 권선구 입북동 801-42,생활방범,4,`2020-04,37.295225,126.958319,POINT (126.958319 37.295225)
...,...,...,...,...,...,...,...
3833,경기도 수원시 권선구 서둔동 17-225,생활방범,4,`2013-12,37.262912,126.993314,POINT (126.993314 37.262912)
3834,경기도 수원시 권선구 서둔동 260-43,생활방범,4,`2013-12,37.278874,126.979325,POINT (126.979325 37.278874)
3835,경기도 수원시 권선구 세류1동 223-2,생활방범,4,`2013-12,37.262486,127.008331,POINT (127.008331 37.262486)
3836,경기도 수원시 권선구 입북동 236-3,생활방범,4,`2013-12,37.294098,126.960498,POINT (126.960498 37.294098)


In [11]:
df_gid

,gid,geometry
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ..."
1,다사49aa19ba,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ..."
2,다사49aa20aa,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ..."
3,다사49ab18ab,"MULTIPOLYGON (((126.92766 37.26177, 126.92764 ..."
4,다사49ab18ba,"MULTIPOLYGON (((126.92764 37.26402, 126.92762 ..."
...,...,...
2085,다사63ba21aa,"MULTIPOLYGON (((127.08822 37.28722, 127.08821 ..."
2086,다사63ba21ab,"MULTIPOLYGON (((127.08821 37.28947, 127.08820 ..."
2087,다사63ba21ba,"MULTIPOLYGON (((127.08820 37.29173, 127.08819 ..."
2088,다사63ba21bb,"MULTIPOLYGON (((127.08819 37.29398, 127.08818 ..."


In [12]:
df_gid['cctv_cnt'] = 0
df_gid

,gid,geometry,cctv_cnt
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",0
1,다사49aa19ba,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ...",0
2,다사49aa20aa,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ...",0
3,다사49ab18ab,"MULTIPOLYGON (((126.92766 37.26177, 126.92764 ...",0
4,다사49ab18ba,"MULTIPOLYGON (((126.92764 37.26402, 126.92762 ...",0
...,...,...,...
2085,다사63ba21aa,"MULTIPOLYGON (((127.08822 37.28722, 127.08821 ...",0
2086,다사63ba21ab,"MULTIPOLYGON (((127.08821 37.28947, 127.08820 ...",0
2087,다사63ba21ba,"MULTIPOLYGON (((127.08820 37.29173, 127.08819 ...",0
2088,다사63ba21bb,"MULTIPOLYGON (((127.08819 37.29398, 127.08818 ...",0


In [13]:
df_gid['cctv_cnt'].unique()

array([0])

In [14]:
# gid 별 cctv 갯수 count
for i in range(0, len(df_cctv_suwon)):
    for j in range(0, len(df_gid)):
        if df_cctv_suwon['geometry'][i].within(df_gid['geometry'][j]):
            df_gid['cctv_cnt'][j] = df_gid['cctv_cnt'][j] + df_cctv_suwon['cctv_cnt'][i]
            print(i, '번 째 작업을 수행중입니다.')

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0 번 째 작업을 수행중입니다.
1 번 째 작업을 수행중입니다.
2 번 째 작업을 수행중입니다.
3 번 째 작업을 수행중입니다.
4 번 째 작업을 수행중입니다.
5 번 째 작업을 수행중입니다.
6 번 째 작업을 수행중입니다.
7 번 째 작업을 수행중입니다.
8 번 째 작업을 수행중입니다.
9 번 째 작업을 수행중입니다.
10 번 째 작업을 수행중입니다.
11 번 째 작업을 수행중입니다.
12 번 째 작업을 수행중입니다.
13 번 째 작업을 수행중입니다.
14 번 째 작업을 수행중입니다.
15 번 째 작업을 수행중입니다.
16 번 째 작업을 수행중입니다.
17 번 째 작업을 수행중입니다.
18 번 째 작업을 수행중입니다.
19 번 째 작업을 수행중입니다.
20 번 째 작업을 수행중입니다.
21 번 째 작업을 수행중입니다.
22 번 째 작업을 수행중입니다.
23 번 째 작업을 수행중입니다.
24 번 째 작업을 수행중입니다.
25 번 째 작업을 수행중입니다.
26 번 째 작업을 수행중입니다.
27 번 째 작업을 수행중입니다.
28 번 째 작업을 수행중입니다.
29 번 째 작업을 수행중입니다.
30 번 째 작업을 수행중입니다.
31 번 째 작업을 수행중입니다.
32 번 째 작업을 수행중입니다.
33 번 째 작업을 수행중입니다.
34 번 째 작업을 수행중입니다.
35 번 째 작업을 수행중입니다.
36 번 째 작업을 수행중입니다.
37 번 째 작업을 수행중입니다.
38 번 째 작업을 수행중입니다.
39 번 째 작업을 수행중입니다.
40 번 째 작업을 수행중입니다.
41 번 째 작업을 수행중입니다.
42 번 째 작업을 수행중입니다.
43 번 째 작업을 수행중입니다.
44 번 째 작업을 수행중입니다.
45 번 째 작업을 수행중입니다.
46 번 째 작업을 수행중입니다.
47 번 째 작업을 수행중입니다.
48 번 째 작업을 수행중입니다.
49 번 째 작업을 수행중입니다.
50 번 째 작업을 수행중입니다.
51 번 째 작업을 수행중입니다.
52 번 째 작업을 수행중입니다.
53 

In [15]:
df_gid['cctv_cnt'].sum()

12083

In [16]:
df_cctv_suwon['cctv_cnt'].sum()

12085

### 표준격자별 보안등 갯수 COUNT

In [20]:
df_sec_light = pd.read_csv('22.4개시도_보안등현황.csv')
df_sec_light

,year,address,securitylight_cnt,lon,lat
0,2016.0,경상남도 진주시 가좌동 1398,1,128.109417,35.159683
1,2010.0,경상남도 진주시 가좌동 1555-1,1,128.125589,35.152041
2,2016.0,경상남도 진주시 가좌동 1937,1,128.107726,35.164435
3,2011.0,경상남도 진주시 가좌동 1977,1,128.111946,35.152001
4,2016.0,경상남도 진주시 가좌동 654-11,1,128.107307,35.162732
...,...,...,...,...,...
218990,NaN,서울특별시 중랑구 상봉동 19-2,1,127.092434,37.602786
218991,NaN,서울특별시 중랑구 상봉동 19-44,1,127.092721,37.602497
218992,NaN,서울특별시 중랑구 상봉동 19-25,1,127.092258,37.602374
218993,NaN,서울특별시 중랑구 상봉동 495-4,1,127.094617,37.602357


In [21]:
df_sec_light['address'].isnull().sum()

10979

In [22]:
df_sec_light.dropna(subset=['address'], inplace=True)
df_sec_light['address'].isnull().sum()

0

In [23]:
df_sec_light[df_sec_light['address'].str.startswith('경기')]

,year,address,securitylight_cnt,lon,lat
171483,2008.0,경기도 수원시 장안구 상광교동 100,1,127.014706,37.331073
171484,2008.0,경기도 수원시 장안구 상광교동 108-1,1,127.016347,37.331537
171485,2008.0,경기도 수원시 장안구 상광교동 113-2,1,127.015894,37.330813
171486,2008.0,경기도 수원시 장안구 상광교동 113-2,1,127.015883,37.330831
171487,2008.0,경기도 수원시 장안구 상광교동 117,1,127.016955,37.329921
...,...,...,...,...,...
185399,2010.0,경기도 수원시 영통구 매탄동 214-16,1,127.048124,37.268471
185400,2010.0,경기도 수원시 영통구 매탄동 214-20,1,127.047645,37.268606
185401,2010.0,경기도 수원시 영통구 매탄동 917-1,1,127.047459,37.268593
185402,2010.0,경기도 수원시 영통구 매탄동 917-3,1,127.048564,37.268385


In [24]:
df_sec_light_suwon = df_sec_light.reindex(index = list(range(171483, 185403 + 1)))
df_sec_light_suwon

,year,address,securitylight_cnt,lon,lat
171483,2008.0,경기도 수원시 장안구 상광교동 100,1.0,127.014706,37.331073
171484,2008.0,경기도 수원시 장안구 상광교동 108-1,1.0,127.016347,37.331537
171485,2008.0,경기도 수원시 장안구 상광교동 113-2,1.0,127.015894,37.330813
171486,2008.0,경기도 수원시 장안구 상광교동 113-2,1.0,127.015883,37.330831
171487,2008.0,경기도 수원시 장안구 상광교동 117,1.0,127.016955,37.329921
...,...,...,...,...,...
185399,2010.0,경기도 수원시 영통구 매탄동 214-16,1.0,127.048124,37.268471
185400,2010.0,경기도 수원시 영통구 매탄동 214-20,1.0,127.047645,37.268606
185401,2010.0,경기도 수원시 영통구 매탄동 917-1,1.0,127.047459,37.268593
185402,2010.0,경기도 수원시 영통구 매탄동 917-3,1.0,127.048564,37.268385


In [25]:
df_sec_light_suwon[df_sec_light_suwon['address'].isnull()]

,year,address,securitylight_cnt,lon,lat
181374,NaN,NaN,NaN,NaN,NaN


In [26]:
df_sec_light_suwon.reset_index(drop=True, inplace=True)

In [27]:
from shapely.geometry import Point, Polygon, LineString
points_sec = []
for i in range(0, len(df_sec_light_suwon)):
    point = Point(df_sec_light_suwon['lon'][i], df_sec_light_suwon['lat'][i])
    points_sec.append(point)

points_sec

 ...]

In [28]:
df_sec_light_suwon['geometry'] = points_sec
df_sec_light_suwon

,year,address,securitylight_cnt,lon,lat,geometry
0,2008.0,경기도 수원시 장안구 상광교동 100,1.0,127.014706,37.331073,POINT (127.014706 37.331073)
1,2008.0,경기도 수원시 장안구 상광교동 108-1,1.0,127.016347,37.331537,POINT (127.016347 37.331537)
2,2008.0,경기도 수원시 장안구 상광교동 113-2,1.0,127.015894,37.330813,POINT (127.015894 37.330813)
3,2008.0,경기도 수원시 장안구 상광교동 113-2,1.0,127.015883,37.330831,POINT (127.015883 37.330831)
4,2008.0,경기도 수원시 장안구 상광교동 117,1.0,127.016955,37.329921,POINT (127.016955 37.329921)
...,...,...,...,...,...,...
13916,2010.0,경기도 수원시 영통구 매탄동 214-16,1.0,127.048124,37.268471,POINT (127.0481241 37.26847100000001)
13917,2010.0,경기도 수원시 영통구 매탄동 214-20,1.0,127.047645,37.268606,POINT (127.047645 37.2686064)
13918,2010.0,경기도 수원시 영통구 매탄동 917-1,1.0,127.047459,37.268593,POINT (127.047459 37.268593)
13919,2010.0,경기도 수원시 영통구 매탄동 917-3,1.0,127.048564,37.268385,POINT (127.0485637 37.26838529999999)


In [29]:
df_gid['securitylight_cnt'] = 0
df_gid

,gid,geometry,cctv_cnt,securitylight_cnt
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",0,0
1,다사49aa19ba,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ...",0,0
2,다사49aa20aa,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ...",0,0
3,다사49ab18ab,"MULTIPOLYGON (((126.92766 37.26177, 126.92764 ...",0,0
4,다사49ab18ba,"MULTIPOLYGON (((126.92764 37.26402, 126.92762 ...",0,0
...,...,...,...,...
2085,다사63ba21aa,"MULTIPOLYGON (((127.08822 37.28722, 127.08821 ...",0,0
2086,다사63ba21ab,"MULTIPOLYGON (((127.08821 37.28947, 127.08820 ...",0,0
2087,다사63ba21ba,"MULTIPOLYGON (((127.08820 37.29173, 127.08819 ...",0,0
2088,다사63ba21bb,"MULTIPOLYGON (((127.08819 37.29398, 127.08818 ...",0,0


In [30]:
# gid 별 보안등 갯수 count
for i in range(0, len(df_sec_light_suwon)):
    for j in range(0, len(df_gid)):
        if df_sec_light_suwon['geometry'][i].within(df_gid['geometry'][j]):
            df_gid['securitylight_cnt'][j] = df_gid['securitylight_cnt'][j] + df_sec_light_suwon['securitylight_cnt'][i]
            print(i, '번 째 작업을 수행중입니다.')

/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0 번 째 작업을 수행중입니다.
1 번 째 작업을 수행중입니다.
2 번 째 작업을 수행중입니다.
3 번 째 작업을 수행중입니다.
4 번 째 작업을 수행중입니다.
5 번 째 작업을 수행중입니다.
6 번 째 작업을 수행중입니다.
7 번 째 작업을 수행중입니다.
8 번 째 작업을 수행중입니다.
9 번 째 작업을 수행중입니다.
10 번 째 작업을 수행중입니다.
11 번 째 작업을 수행중입니다.
12 번 째 작업을 수행중입니다.
13 번 째 작업을 수행중입니다.
14 번 째 작업을 수행중입니다.
15 번 째 작업을 수행중입니다.
16 번 째 작업을 수행중입니다.
17 번 째 작업을 수행중입니다.
18 번 째 작업을 수행중입니다.
19 번 째 작업을 수행중입니다.
20 번 째 작업을 수행중입니다.
21 번 째 작업을 수행중입니다.
22 번 째 작업을 수행중입니다.
23 번 째 작업을 수행중입니다.
24 번 째 작업을 수행중입니다.
25 번 째 작업을 수행중입니다.
26 번 째 작업을 수행중입니다.
27 번 째 작업을 수행중입니다.
28 번 째 작업을 수행중입니다.
29 번 째 작업을 수행중입니다.
30 번 째 작업을 수행중입니다.
31 번 째 작업을 수행중입니다.
32 번 째 작업을 수행중입니다.
33 번 째 작업을 수행중입니다.
34 번 째 작업을 수행중입니다.
35 번 째 작업을 수행중입니다.
36 번 째 작업을 수행중입니다.
37 번 째 작업을 수행중입니다.
38 번 째 작업을 수행중입니다.
39 번 째 작업을 수행중입니다.
40 번 째 작업을 수행중입니다.
41 번 째 작업을 수행중입니다.
42 번 째 작업을 수행중입니다.
43 번 째 작업을 수행중입니다.
44 번 째 작업을 수행중입니다.
45 번 째 작업을 수행중입니다.
46 번 째 작업을 수행중입니다.
47 번 째 작업을 수행중입니다.
48 번 째 작업을 수행중입니다.
49 번 째 작업을 수행중입니다.
50 번 째 작업을 수행중입니다.
51 번 째 작업을 수행중입니다.
52 번 째 작업을 수행중입니다.
53 

In [31]:
df_gid['securitylight_cnt'].sum()

14362

In [32]:
df_sec_light_suwon['securitylight_cnt'].sum()

14367.0

In [33]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link(df, title = "Download CSV file", filename = "security_facility.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_gid)

### 표준격자별 가로등 갯수 COUNT

In [34]:
GetCompasData('SBJ_2109_001', '21', '21.수원시_가로등현황.csv')

In [3]:
df_street_light = pd.read_csv('21.수원시_가로등현황.csv')
df_street_light

,sgg_nm,road_addr,jibun_addr,lat,lon,install_year,install_type,manage_dept,manage_dept_pn,base_date
0,수원시,NaN,경기도 수원시 장안구 연무동 산 1-4,37.301144,127.031677,2011,전용주,도로관리과,031-228-4309,2020-08-21
1,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.300966,127.031689,2011,전용주,도로관리과,031-228-4309,2020-08-21
2,수원시,NaN,경기도 수원시 장안구 연무동 4-36,37.301056,127.032005,2011,전용주,도로관리과,031-228-4309,2020-08-21
3,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.301155,127.032377,2011,전용주,도로관리과,031-228-4309,2020-08-21
4,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.301229,127.032653,2011,전용주,도로관리과,031-228-4309,2020-08-21
...,...,...,...,...,...,...,...,...,...,...
24458,수원시,NaN,경기도 수원시 팔달구 화서동 720-3,37.287592,126.979577,2018,전용주,도로관리과,031-228-4309,2020-08-21
24459,수원시,NaN,경기도 수원시 팔달구 화서동 720-3,37.287827,126.979470,2018,전용주,도로관리과,031-228-4309,2020-08-21
24460,수원시,NaN,경기도 수원시 팔달구 화서동 720-2,37.288092,126.979346,2018,전용주,도로관리과,031-228-4309,2020-08-21
24461,수원시,NaN,경기도 수원시 팔달구 화서동 720-2,37.288322,126.979247,2018,전용주,도로관리과,031-228-4309,2020-08-21


In [4]:
df_street_light['streetlight_cnt'] = 1

In [5]:
df_street_light.head()

,sgg_nm,road_addr,jibun_addr,lat,lon,install_year,install_type,manage_dept,manage_dept_pn,base_date,streetlight_cnt
0,수원시,NaN,경기도 수원시 장안구 연무동 산 1-4,37.301144,127.031677,2011,전용주,도로관리과,031-228-4309,2020-08-21,1
1,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.300966,127.031689,2011,전용주,도로관리과,031-228-4309,2020-08-21,1
2,수원시,NaN,경기도 수원시 장안구 연무동 4-36,37.301056,127.032005,2011,전용주,도로관리과,031-228-4309,2020-08-21,1
3,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.301155,127.032377,2011,전용주,도로관리과,031-228-4309,2020-08-21,1
4,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.301229,127.032653,2011,전용주,도로관리과,031-228-4309,2020-08-21,1


In [6]:
from shapely.geometry import Point, Polygon, LineString
points_st = []
for i in range(0, len(df_street_light)):
    point = Point(df_street_light['lon'][i], df_street_light['lat'][i])
    points_st.append(point)

points_st

 ...]

In [7]:
df_street_light['geometry'] = points_st
df_street_light

,sgg_nm,road_addr,jibun_addr,lat,lon,install_year,install_type,manage_dept,manage_dept_pn,base_date,streetlight_cnt,geometry
0,수원시,NaN,경기도 수원시 장안구 연무동 산 1-4,37.301144,127.031677,2011,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (127.031677 37.301144)
1,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.300966,127.031689,2011,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (127.031689 37.300966)
2,수원시,NaN,경기도 수원시 장안구 연무동 4-36,37.301056,127.032005,2011,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (127.032005 37.301056)
3,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.301155,127.032377,2011,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (127.032377 37.301155)
4,수원시,NaN,경기도 수원시 장안구 연무동 4-25,37.301229,127.032653,2011,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (127.032653 37.301229)
...,...,...,...,...,...,...,...,...,...,...,...,...
24458,수원시,NaN,경기도 수원시 팔달구 화서동 720-3,37.287592,126.979577,2018,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (126.979577 37.287592)
24459,수원시,NaN,경기도 수원시 팔달구 화서동 720-3,37.287827,126.979470,2018,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (126.97947 37.287827)
24460,수원시,NaN,경기도 수원시 팔달구 화서동 720-2,37.288092,126.979346,2018,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (126.979346 37.288092)
24461,수원시,NaN,경기도 수원시 팔달구 화서동 720-2,37.288322,126.979247,2018,전용주,도로관리과,031-228-4309,2020-08-21,1,POINT (126.979247 37.28832199999999)


In [8]:
df_gid['streetlight_cnt'] = 0
df_gid

,gid,geometry,streetlight_cnt
0,다사49aa19ab,"MULTIPOLYGON (((126.92477 37.27077, 126.92475 ...",0
1,다사49aa19ba,"MULTIPOLYGON (((126.92475 37.27302, 126.92473 ...",0
2,다사49aa20aa,"MULTIPOLYGON (((126.92472 37.27753, 126.92470 ...",0
3,다사49ab18ab,"MULTIPOLYGON (((126.92766 37.26177, 126.92764 ...",0
4,다사49ab18ba,"MULTIPOLYGON (((126.92764 37.26402, 126.92762 ...",0
...,...,...,...
2085,다사63ba21aa,"MULTIPOLYGON (((127.08822 37.28722, 127.08821 ...",0
2086,다사63ba21ab,"MULTIPOLYGON (((127.08821 37.28947, 127.08820 ...",0
2087,다사63ba21ba,"MULTIPOLYGON (((127.08820 37.29173, 127.08819 ...",0
2088,다사63ba21bb,"MULTIPOLYGON (((127.08819 37.29398, 127.08818 ...",0


In [9]:
# gid 별 가로등 갯수 count
for i in tqdm(range(0, len(df_street_light))):
    for j in range(0, len(df_gid)):
        if df_street_light['geometry'][i].within(df_gid['geometry'][j]):
            df_gid['streetlight_cnt'][j] = df_gid['streetlight_cnt'][j] + df_street_light['streetlight_cnt'][i]
            

  0%|          | 0/24463 [00:00<?, ?it/s]/opt/app-root/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 24463/24463 [2:17:47<00:00,  2.96it/s]  


In [10]:
df_gid['streetlight_cnt'].sum()

24449

In [12]:
df_street_light['streetlight_cnt'].sum()

24463

In [ ]:
df.to_csv("test.csv")

In [13]:
import base64
import pandas as pd
from IPython.display import HTML

def create_download_link(df, title = "Download CSV file", filename = "street_light_cnt.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(df_gid)